In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import time
import re
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm_notebook

In [22]:
Urls = pd.read_csv('url_total.csv', index_col = 0)

In [23]:
Urls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25742 entries, 0 to 25741
Data columns (total 2 columns):
cat2    25742 non-null int64
url     25742 non-null object
dtypes: int64(1), object(1)
memory usage: 603.3+ KB


In [24]:
top = ['12015003002','12015003003','12015003004',
       '12015003005','12015003006','12015003007']
outer = ['12015001001', '12015004001', '12015004002', '12015004003', '12015004004']
bottom = ['12015009001', '12015009002', '12015009003', '12015009005', '12015009004']
shoes = ['12016013001001', '12016013003001', '12016013007001', '12016013001002', '12016013002001',
         '12016013004004', '12016013003002', '12016013002002', '12016013004005', '12016013001003',
         '12016013004002', '12016013003003', '12016013001004', '12016013005', '12016013004003',
         '12016013003004', '12016013001005', '12016013006', '12016013003005', '12016013007003',
         '12016013002003', '12016013004001', '12016013008', '12016013009', '12016013001006',
         '12016013003006', '12016013001007', '12016013003007', '12016013007004', '12016013007002',
         '12016013010', '12016013001008', '12016013001009','12016013004006']
bags = ['12016021001', '12016021002', '12016021003', '12016001001',
        '12016001004001', '12016001002', '12016001003', '12016001004002',
        '12016021004', '12016001004003', '12016001004004', '12016021005',
        '12016021006', '12016021007', '12016021008', '12016001004006',
        '12016001005', '12016001006', '12016001007', '12016001008', '12016001009']

# 드라이버 위치 바꿔주기 ↓

In [34]:
driver = webdriver.Chrome('/Users/hwii/Downloads/chromedriver_mac64/chromedriver')

In [35]:
root = """https://swindow.naver.com"""

In [7]:
def click_item(variable):
    driver.execute_script("return arguments[0].scrollIntoView();", variable)
    time.sleep(0.5)
    ActionChains(driver).click(variable).perform()
    time.sleep(0.5)

# !! 테스트 !!
## 다섯개만 일단 먼저 돌려보기 

In [8]:
reviews = []
ratings = []
error_url = []
product_id = []

# 임의로 12000번 이후로 일단 설정해뒀어! 나는 1200개까지 지금 돌려놓음 ㅎㅎ
for url in tqdm_notebook(Urls['url'][12000:12005]):
    driver.get(root+url)
    time.sleep(0.5)
    prod_id = re.split('\?|/', url)[2]
    try:
        # 0. 리뷰 가져올 준비
        # 리뷰 섹션 클릭하기
        showreview = driver.find_element_by_xpath("""//*[@id="content"]/div[1]/div[2]/ul/li[2]/a""")
        showreview.click()

        # 프리미엄 리뷰 더보기
        showmore = driver.find_elements_by_class_name('button_more')
        for button in showmore:
            click_item(button)
            
        # 1. 리뷰 가져오기
        # 1-1. 프리미엄 리뷰 (제목), 일반 리뷰 (본문) 소스 구분 없이 다 찾기
        rev = driver.find_elements_by_class_name("header_review_comment")
        rev_total_cnt = len(rev) # 전체 리뷰 갯수 저장
        
        # 1-2. 프리미엄 리뷰 본문만 따로 찾기
        prm_rev_location = driver.find_elements_by_class_name("area_full_article");
        prm_cnt = len(prm_rev_location) # 프리미엄 리뷰 갯수 저장
        
        # 1-3. 1-1에서 찾은 프리미엄 리뷰 제목이랑 본문 합칠건데, 리스트 길이가 안 맞으니까 맞춰주기
        prm_extend = [''] * (len(rev) - len(prm_rev_location))
        prm_rev_contents = [item.text.split('\n판매자 ')[0] for item in prm_rev_location] + prm_extend
        
        # 1-4. 일반 리뷰 소스에서 본문 text 가져오기
        rev_content = [item.text for item in rev]
        
        # 1-5. 프리미엄 : 제목이랑 본문 합치기, 일반 리뷰
        total_content = [rev_content[i] + ' ' + prm_rev_contents[i] for i in range(rev_total_cnt)]
        
        # 1-6. 리뷰 total_content에 저장!
        reviews.extend(total_content)
        
        # 2. 만족도 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        rate_loc = soup.find_all('div', 'col_label') # 만족도 소스 찾기
        rate = [r.get_text().strip() for r in rate_loc] # 텍스트만 따로 저장
        ratings.extend(rate) # rating 에 저장
        
        # 3. 상품 번호 저장 (리뷰의 수만큼! 같은 상품의 리뷰임을 알아야하니까)
        product_id.extend([prod_id] * rev_total_cnt)

    except:
        # 에러난 애들 ㅠㅠ
        product_id.append(prod_id) # 상품 번호 기록
        reviews.append(None) # 리뷰, 만족도에는 None
        ratings.append(None) 
        error_url.append(url) # 따로 url 저장

## 확인사항

In [9]:
# 1. 길이가 서로 맞는지
len(reviews), len(product_id), len(ratings)

(7, 7, 7)

In [10]:
# 2. 잘 나왔나 한번 보기
reviews

['이쁩니다. ^^ ',
 '구두가 많이구겨져 와서 기분이 안좋았지만 색깔은 이뻐요 그리고 배송은 빨라요 ',
 None,
 None,
 None,
 '무리하게 빠른배송 요구햇는데 약속지켜주셔서 넘 감사했어요! 가죽상태도 넘좋고 완전 만족합니다~~^^ ',
 '너무 맘에들어요 발도 오래신어도 안아프네요 ']

# !! 여기부터 진짜 코드 !!
## *실행 전 수정사항*
## 1. for 문 시작되는 곳에 어느 부분 데이터 돌릴지 설정하기
## 2. 맨 아랫줄에 저장할 csv 파일 이름 바꾸기
### 아직 상품 태그는 안 가져옴 ㅠㅠ 일단 리뷰만.. 나중에 에러 안난 링크랑 상품평 적어도 n개 이상 등등 각종 기준 만족하는 애들만 추려서 상품 정보 가져오자!

In [36]:
reviews = []
ratings = []
error_url = []
product_id = []

# 임의로 12000번 이후로 일단 설정해뒀어! 나는 1200개까지 지금 돌려놓음 ㅎㅎ
for url in tqdm_notebook(Urls['url'][24722:]):
    driver.get(root+url)
    time.sleep(0.5)
    prod_id = re.split('\?|/', url)[2]
    try:
        # 0. 리뷰 가져올 준비
        # 리뷰 섹션 클릭하기
        #time.sleep(2)
        showreview = driver.find_element_by_xpath("""//*[@id="content"]/div[1]/div[2]/ul/li[2]/a""")
        #showreview.click()
        click_item(showreview)

        # 프리미엄 리뷰 더보기
        showmore = driver.find_elements_by_class_name('button_more')
        for button in showmore:
            click_item(button)
            
        # 1. 리뷰 가져오기
        # 1-1. 프리미엄 리뷰 (제목), 일반 리뷰 (본문) 소스 구분 없이 다 찾기
        rev = driver.find_elements_by_class_name("header_review_comment")
        rev_total_cnt = len(rev) # 전체 리뷰 갯수 저장
        
        # 1-2. 프리미엄 리뷰 본문만 따로 찾기
        prm_rev_location = driver.find_elements_by_class_name("area_full_article");
        prm_cnt = len(prm_rev_location) # 프리미엄 리뷰 갯수 저장
        
        # 1-3. 1-1에서 찾은 프리미엄 리뷰 제목이랑 본문 합칠건데, 리스트 길이가 안 맞으니까 맞춰주기
        prm_extend = [''] * (len(rev) - len(prm_rev_location))
        prm_rev_contents = [item.text.split('\n판매자 ')[0] for item in prm_rev_location] + prm_extend
        
        # 1-4. 일반 리뷰 소스에서 본문 text 가져오기
        rev_content = [item.text for item in rev]
        
        # 1-5. 프리미엄 : 제목이랑 본문 합치기, 일반 리뷰
        total_content = [rev_content[i] + ' ' + prm_rev_contents[i] for i in range(rev_total_cnt)]
        
        # 1-6. 리뷰 total_content에 저장!
        reviews.extend(total_content)
        
        # 2. 만족도 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        rate_loc = soup.find_all('div', 'col_label') # 만족도 소스 찾기
        rate = [r.get_text().strip() for r in rate_loc] # 텍스트만 따로 저장
        ratings.extend(rate) # rating 에 저장
        
        # 3. 상품 번호 저장 (리뷰의 수만큼! 같은 상품의 리뷰임을 알아야하니까)
        product_id.extend([prod_id] * rev_total_cnt)

    except:
        # 에러난 애들 ㅠㅠ
        product_id.append(prod_id) # 상품 번호 기록
        reviews.append(None) # 리뷰, 만족도에는 None
        ratings.append(None) 
        error_url.append(url) # 따로 url 저장
        
# 데이터 프레임으로 만들어서 csv 파일로 저장
df = pd.DataFrame({'Review': reviews, 'Rating': ratings, 'Id': product_id})
df.to_csv('Reviews_24722_end.csv', encoding = 'utf-8') # 어떤 부분인지 이름 바꿔주기

In [30]:
len(reviews), len(product_id), len(ratings)

(46393, 46393, 46393)

In [32]:
df.head()

,Id,Rating,Review
0,2298929319,추천,컬러가 이뻐요^^ 세일상품 저렴하게 잘 샀어요^^ 생각보다 도톰하니 따뜻하고 좋은데...
1,2298929319,적극추천,겨울 따뜻하게 보냈어용 빨간색 사고 넘 맘에 들어서 아이보리색도 재구매 하게 되었어...
2,2298929319,적극추천,부들부들 이뻐요 니트야 원래 보풀일어나는법 이가격에 득템함 엘로우 넘 이뻐요 깔별로...
3,2298929319,추천,"이뻐요. 보라색이너무이ㅃㄷ네요.보라색했는데너무마음에들고좋아요.좋아요.,그리고 좋아요..."
4,2298929319,적극추천,밝은 아이보리 두툼하고 부들부들하고~ 행사가있어서 주문하게됐지만 돌아오는 가을부터 ...
